In [1]:
# In this file.ipynb, we will the following tasks:

# (1): Set up all necessary input parameters for the network design of the most general QS G/G/K with variable capacity
# (2): Define the fractional integer nonlinear programming, including the objective function, 
    # decision variables and constraints
# (3): Following the pseudo-code with 5 steps, to reformulate the original fractional integer nonlinear programming
    # equivalently as an MILP problem
# (4): solve the MILP problem by Gurobi solver
    
# Remark: This file.py works after the prediction step is done. Otherwise, we will assign randomly deterministic values
    # for the input parameters to run the program

# Import all the useful python libraries here, which will be used later

In [2]:
# %reset -f

In [3]:
import numpy as np # import the well-known package numpy for computation in Python
import matplotlib.pyplot as plt # import the well-known package matplotlib for visualization in Python
import math # import the well-known package math for some math calculations, i.e., factorial
import itertools  # this package is to list all subsets  of a given set in Python
import pandas as pd
import os
from itertools import product

# Import Data - Remark Here: Update the directory when change to Linux

In [4]:
os.chdir('C:\\Users\\33663\\Queuing project data')
data_folder = 'C:\\Users\\33663\\Queuing project data' # use for my window computer
# data_folder = 'home/wenbo/nguyen' # use for linux computer
#data_folder = os.getcwd() # use for linux computer

In [5]:
import csv
import random

# Specify the input and output file paths
input_file = 'OverdoseUpdated.csv'
output_file = 'Customer150Test.csv'

# Number of rows to select randomly
num_rows_to_select = 100

# Open the input CSV file and read its contents
with open(input_file, 'r', newline='') as csv_in:
    reader = csv.reader(csv_in)
    header = next(reader)  # Read the header
    rows = list(reader)    # Read the remaining rows into a list

# Randomly select num_rows_to_select rows
selected_rows = random.sample(rows, num_rows_to_select)

# Write the selected rows to the output CSV file
with open(output_file, 'w', newline='') as csv_out:
    writer = csv.writer(csv_out)
    writer.writerow(header)  # Write the header
    writer.writerows(selected_rows)

print("Random rows have been selected and written to", output_file)

Random rows have been selected and written to Customer150Test.csv


In [6]:
input_file = 'BaseStations.csv'
output_file = 'Facility20Test.csv'

# Number of rows to select randomly
num_rows_to_select = 5

# Open the input CSV file and read its contents
with open(input_file, 'r', newline='') as csv_in:
    reader = csv.reader(csv_in)
    header = next(reader)  # Read the header
    rows = list(reader)    # Read the remaining rows into a list

# Randomly select num_rows_to_select rows
selected_rows = random.sample(rows, num_rows_to_select)

# Write the selected rows to the output CSV file
with open(output_file, 'w', newline='') as csv_out:
    writer = csv.writer(csv_out)
    writer.writerow(header)  # Write the header
    writer.writerows(selected_rows)

print("Random rows have been selected and written to", output_file)

Random rows have been selected and written to Facility20Test.csv


# Load Data

In [7]:
facility=pd.read_csv(data_folder + '/Facility20Test.csv')
customer_loc=pd.read_csv(data_folder + '/Customer150Test.csv')
overdose=pd.read_csv(data_folder + '/Overdose.csv')

In [8]:
facility.columns = [col.lower() for col in facility.columns]
customer_loc.columns = [col.lower() for col in customer_loc.columns]
overdose_base = pd.merge(left = customer_loc.assign(key = 1), right = facility.assign(key = 1), on = 'key', suffixes = ('_demand','_base')).drop('key', axis = 1)

In [9]:
n=len(facility)
# n = |J| is the number of candidate locations for facility. The value of n is determined from prediction step. 
# For now, we temporarily assign a deterministic value for n to test the code.

In [10]:
m=len(customer_loc)
# m = |I| is the number of customer or demand points. The value of m is also determined from prediction step.
# For now, we temporarily assign a deterministic value for m to test the code.

In [11]:
# Define I set of customer locations
I=[] 
for i in range(m):
    I.append(i)

In [12]:
# Define  J set of candidate facility locations
J=[]
for j in range(n):
    J.append(j)

In [13]:
# customer['receivedtime'] = pd.to_datetime(customer['receivedtime'])
# customer['demand_year'] = customer['receivedtime'].dt.year
# customer['demand_month'] = customer['receivedtime'].dt.month
# customer['mon_yr'] = customer['receivedtime'].dt.strftime('%Y-%m')
# customer['qrt_yr'] = customer['receivedtime'].dt.year.astype(str) + '-' + customer['receivedtime'].dt.quarter.astype(str)
# customer['location'] = customer.apply(lambda x: (x['latitude'], x['longitude']), axis = 1)
# customer = customer.sort_values(by = 'receivedtime')
# occ = customer.location.value_counts().rename_axis('location').reset_index(name = 'occurence') # calculate number of demands for each demand location
# customer_loc = pd.merge(left = customer, right = occ, how = 'left', on = 'location')
# customer_loc = customer_loc.sort_values(by ='receivedtime').drop_duplicates(subset = 'location', keep = 'first').reset_index(drop = True) # keep only one record for each location because what we need is the occurence
# customer_loc.drop('location', inplace=True, axis=1) 
takeoff_land_time = 10 / 3600 # in hour
v_drone = 27.8 * 3600 # drone flight speed in m/hour
train_max_flight_time = 25 / 2 / 60 - takeoff_land_time # in hour 
radius = v_drone * train_max_flight_time # flight radius
non_travel_service = 1500 / 3600 # in hour

In [14]:
def cal_dist(row):
    R = 6378 * 1000
    delta_lat = (row['latitude_base'] - row['latitude_demand']) * np.pi / 180
    delta_log = (row['longitude_base'] - row['longitude_demand']) * np.pi / 180
    a = np.sin(delta_lat / 2) ** 2 + np.cos(row['latitude_demand'] * np.pi / 180) * np.cos(row['latitude_base'] * np.pi / 180) * np.sin(delta_log / 2) ** 2
    d = 2 * R * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return d

In [15]:
overdose_base['distance'] = overdose_base.apply(lambda x: cal_dist(x), axis = 1)
overdose_base['is_within_r'] = overdose_base.distance.apply(lambda x: 0 if x > radius else 1)
overdose_base['i_j_ind'] = overdose_base.apply(lambda x: (x['id_ohca'], x['id_bases']), axis = 1)
distanceinfo = pd.Series(overdose_base['distance'].values, index = overdose_base['i_j_ind']).to_dict()
check = pd.Series(overdose_base['is_within_r'].values, index = overdose_base['i_j_ind']).to_dict()
overdose_base.groupby('id_ohca')['is_within_r'].max().mean()
demands = list(customer_loc.id_ohca.values)
drone_bases = list(facility.id_bases.values)
demand_base = list(product(demands, drone_bases))
demand_base_reduce = [(demands.index(each[0]),drone_bases.index(each[1])) for each in demand_base if check[each[0], each[1]] == 1]
demand_base_reducetest = [each for each in list(product(demands, drone_bases)) if check[each[0], each[1]] == 1]
listIj=[]
for j in J:
    listIj.append([])
listJi=[]
for i in I:
    listJi.append([])
for each in demand_base:
    if check[each[0], each[1]] == 1:
        listIj[drone_bases.index(each[1])].append(demands.index(each[0]))
        listJi[demands.index(each[0])].append(drone_bases.index(each[1]))

# distance between a facility location (base) and a customer location (demand) 

In [16]:
def dist(facility,customer_loc,i,j):
    R = 6378 * 1000
    delta_lat = (facility['latitude'][j] - customer_loc['latitude'][i]) * np.pi / 180
    delta_log = (facility['longitude'][j] - customer_loc['longitude'][i]) * np.pi / 180
    a = np.sin(delta_lat / 2) ** 2 + np.cos(customer_loc['latitude'][i] * np.pi / 180) * np.cos(facility['latitude'][j] * np.pi / 180) * np.sin(delta_log / 2) ** 2
    d = 2 * R * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return d

# Period of Time

In [17]:
overdose['ReceivedTime'] = pd.to_datetime(overdose['ReceivedTime'])

In [18]:
data_hrs = (overdose.ReceivedTime.max() - overdose.ReceivedTime.min()).days * 24 # calculate hours covering the entire training data

# Input parameters for ND^{G/G/K-var} from prediction step

In [19]:
# In this part, we will set up all the necessary input parameters for the network design model. The parameters are:

# the number of candidate facility locations   
# the number of customer locations       
# the maximum capacity at each facility location        
# the maximum number of locations, which can be set up facility                 
# the total number of facilities (i.e. the total capacity)
# average time needed for mobile servers to move on scences from j to i (in drone paper, this is the ratio of geographical
   # distance between i,j and the speed of drones at j)
# square coefficient of variation of inter-arrival times of demands from i to j, which depends on expected value and 
   # second moment of the random inter-arrival times
# expected value of random service time needed for servers at j to support customers at i
# second moment of random service time needed for servers at j to support customers at i

In [20]:
BarW = 2
# BarW is the maximum allowed capacity of every facility location. The value of BarW is determined directly here.

In [21]:
BarL = 4
# BarL is the limit on the number of facility locations that can be set up. The value of BarL is determined directly here. 

In [22]:
BarT = 6
# BarT is the total number of facilities (i.e. drones). The value of BarT is determined directly here.

$$\mbox{arrival rates} \ \ \lambda_{i} \ \ \mbox{defined here}$$

In [23]:
lamda=np.zeros(shape=(m)) # creating a column vector with m elements
for i in I:
    lamda[i] = int(customer_loc['occurence'][i]) / data_hrs * 10 ** 2 * 4

$$\mbox{travel times} \ \ t_{ij} \ \ \mbox{defined here} = \frac{\mbox{distance}(i,j)}{\mbox{drone_speed}}$$

In [24]:
t=np.zeros(shape=(m,n)) # creating a matrix, size m x n
for i in I:
    for j in J:
        t[i,j]=dist(facility,customer_loc,i,j) / v_drone
# t_{ij} is the time needed for mobile servers to move from location j to location i to support customers.
# For now, we temporarily assign a deterministic value for t to test the code.

$$\mathbb{E}[T_{ij}] = \frac{1}{\mbox{lamda}[i]}$$

In [25]:
MeanInterArrivalTime=np.zeros(shape=(m)) # creating a column vector with m elements
for i in I:
    MeanInterArrivalTime[i] = 1 / lamda[i]

$$\mathbb{E}[T^2_{ij}] = \mathbb{E}[T_{ij}]^2 + \mbox{a small random quantity}$$

In [26]:
SecondInterArrivalTime=np.zeros(shape=(m)) # creating a column vector with m elements
for i in I:
    SecondInterArrivalTime[i] = 2 * MeanInterArrivalTime[i] ** 2 + 0.2

$$c_{T_{ij}} = \frac{\sqrt{\mathbb{E}[T^2_{ij}]-\mathbb{E}[T_{ij}]^2}}{\mathbb{E}[T_{ij}]}$$

$$\mbox{Here  c2T} \ \ = c^2_{T_{ij}}, \ \ \mbox{see Section Preliminaries on Queuing Metric}$$

In [27]:
c2T = np.zeros(shape=(m,n)) # creating a matrix, size m x n
for i in I:
    for j in J:
        c2T[i,j]=(SecondInterArrivalTime[i] - MeanInterArrivalTime[i]**2) / (MeanInterArrivalTime[i]**2)
# c2T[i,j] is the square coefficient of variation of the random inter-arrival time of demands from i to j.
# c2T[i,j] = \sqrt{E[T^2_{ij}]-E[T_{ij}]^2} / E[T_{ij}]
# c2T is determined by the expected value of inter-arrival times E[T_{ij}] and the second moment of inter-arrival
# time E[T_{ij}^2], which are determined from the prediction step. 
# Note that c2T = 1 in case of Poisson arrivals, but here is distribution-free inter-arrival times.
# For now, we temporarily assign a deterministic value for c2T to test the code.

$$\mbox{Here is the def of mean service times MeanS[i,j]} = \ \  \ \mathbb{E}[S_{ij}] = \frac{t_{ij}}{2} + \mbox{non travel service}$$

In [28]:
MeanS = np.zeros(shape=(m,n)) # creating a matrix, size m x n
for i in I:
    for j in J:
        MeanS[i,j] = t[i,j] * 2 + non_travel_service
# MeanS[i,j] is the expected value of service time S_{ij}, needed for servers at location j to support customers
# at location i. MeanS is determined from prediction step, but we temporarily assign a deterministic value
# for MeanS to test the code for now.

$$\mbox{Here is the def of second moment of service times SecondMS[i,j]} = \ \  \ \mathbb{E}[S^2_{ij}] = 2 \mathbb{E}[S_{ij}]^2 + \mbox{a small random quantity}$$

In [29]:
SecondMS = np.zeros(shape=(m,n)) # creating a matrix, size m x n
for i in I:
    for j in J:
        SecondMS[i,j] = 2 * MeanS[i,j] ** 2 + 0.2
# SecondMS[i,j] is the second value of service time S_{ij}, needed for servers at location j to support customers
# at location i. SecondMS is determined from prediction step, but we temporarily assign a deterministic value
# for SecondMS to test the code for now.

In [30]:
# Now, all the input parameters of the network design model for G/G/K-var are set up. Now, we determine the generic
# formulation for ND^{G,G,K-var} by creating the model, define the decision variables, constraints and objective function
# The solver Gurobi will be used

# Import the Gurobi solver in Python

In [31]:
import gurobipy as gp # import Gurobi solver for Python
from gurobipy import GRB # import the package GRB for attributes, params and constants in Gurobi solver

params = {
"WLSACCESSID": '76b76ad6-4bc4-4abe-a326-c0aab288b806',
"WLSSECRET": 'd8a0b6cf-7cda-401d-9be5-62881f316f00',
"LICENSEID": 2471257,
} # import the key license for Gurobi

env = gp.Env(params=params) # define the Gurobi environment

Set parameter Username
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2471257
Academic license 2471257 - for non-commercial use only - registered to ho___@gwu.edu


# Define the fractional integer nonlinear optimization problem ND^{G/G/K-var}

In [32]:
model = gp.Model(env=env) # Create the model within the Gurobi environment

In [33]:
# We now add main decision variables in the model, including location-allocation-capacity

In [34]:
x=model.addVars(n, vtype=GRB.BINARY,name='x') # define location binary variable x in the model, x_j = 1 if we set up 
    # facility at location j, x_j=0, otherwise
y=model.addVars(m,n, vtype=GRB.BINARY,name='y') # define assignment binary variable y in the model, y_{ij} = 1 if location 
 #  j is assigned to support customers at location i, y_{ij} = 0, otherwise
alpha=model.addVars(n, BarW + 1, vtype=GRB.BINARY,name='alpha') # introduce auxiliary binary variable allpha in the model
model.update()

$$\mbox{Arrival rate} \ \ Cumulate\_Arrival(y,j) \ \ \mbox{defined endogenously by} \ \ \ \sum_{i \in I} \lambda_{i} y_{ij}$$

In [35]:
# Definition of \sum_{i \in I} \lamda_{i} y_{ij}
def Cumulate_Arrival(y,j): 
    SD=0
    for l in listIj[j]:
        SD = SD + lamda[l] * y[l,j]
    return SD

$$Cumulate\_SquareCV(y,j) \ \ = \sum_{i \in I} \lambda_{i} y_{ij} c^2_{T_{ij}}$$

In [36]:
# Definition of \sum_{i \in I} \lamda_{i} y_{ij} c^2_{T_{ij}}
def Cumulate_SquareCV(y,j):
    SD=0
    for l in listIj[j]:
        SD = SD + lamda[l] * c2T[l,j] * y[l,j] 
    return SD

$$Cumulate\_MeanService(y,j) \ \ = \sum_{i \in I} \lambda_{i} y_{ij} E[S_{ij}]$$

In [37]:
# Definition of \sum_{i \in I} \lamda_{i} y_{ij} E[S_{ij}]
def Cumulate_MeanService(y,j):
    SD=0
    for l in listIj[j]:
        SD = SD + lamda[l] * MeanS[l,j] * y[l,j] 
    return SD

$$Cumulate\_SecondService(y,j) \ \ = \sum_{i \in I} \lambda_{i} y_{ij} E[S^2_{ij}]$$

In [38]:
# Definition of \sum_{i \in I} \lamda_{i} y_{ij} E[S^2_{ij}]
def Cumulate_SecondService(y,j):
    SD=0
    for l in listIj[j]:
        SD = SD + lamda[l] * SecondMS[l,j] * y[l,j] 
    return SD

$$fractional\_objective\_function(y,w) \ \ \mbox{is the objective function of } \ \ \mathbf{ND}^{G,G,K^{(v)}} \ \ $$

In [39]:
def fractional_objective_function(y,w): # we define the fractional objective function of ND^{G/G/K-var}
    SD=0
    for i in I:
        for j in listJi[i]:
            if w[j]==0: # Note that at w_j=0, the fractional response time is undefined. So we only take into account w_j>0
                SD=SD
            else:
                term1 = Cumulate_SquareCV(y,j) / Cumulate_Arrival(y,j) + Cumulate_Arrival(y,j) * Cumulate_SecondService(y,j) / (Cumulate_MeanService(y,j)**2) - 1
                term2 = 1 / (2 * Cumulate_Arrival(y,j) * math.factorial(int(w[j])-1))
                term3 = 1 / ((int(w[j])-Cumulate_MeanService(y,j))**2)
                term4 = Cumulate_MeanService(y,j)**(int(w[j])+1)
                test1=0
                for h in range(int(w[j])):
                    test1=test1 + (Cumulate_MeanService(y,j)**h) / math.factorial(h)
                term5=test1 + (Cumulate_MeanService(y,j)**int(w[j])) / (math.factorial(int(w[j])-1)*(int(w[j])-Cumulate_MeanService(y,j)))
                term6=float(lamda[i]) * y[i,j] / float(sum(lamda))
                SD = SD + (t[i,j] + term1 * term2 * term3 * term4 * term5) * term6
    return SD

In [40]:
#  So now, our fractional integer nonlinear optimization problem is ready.
#  The problem is to minimize fractional_objective_function with respect to location variable x, assignment variable y
#  and capacity variable w, subject to a chance constraint and (x,y,w) belongs to \mathcal{F}_{K-var}. We will add 
#  these two constraints into the model later. Now, we move to 5 Steps of the Algorithm "Linearization of the Fractional
#  Objective Function"

# Linearization of the Fractional Objective Function

In [41]:
# We will follow 5 Steps of the pseudo-code

# Step 1: Fractional Nonlinear 0-1 Reformulation via Binary Expansion

$$\mbox{Add} \ \ x_j \le \sum_{u=1}^{\bar{w}}u\alpha_{ju} \le \bar{w} x_j, \ \forall \ j \in J \ \ \mbox{into the model} $$

In [42]:
# Add constraints 5.1c into the model (including 2 x n constraints)

In [43]:
for j in J:
    model.addConstr(x[j] <= gp.quicksum(u*alpha[j,u] for u in range(1, BarW + 1)), name='5.1cLeftSide'+str(j)+'th')
    model.addConstr(gp.quicksum(u*alpha[j,u] for u in range(1, BarW + 1)) <= BarW * x[j], name='5.1cRightSide'+str(j)+'th')
model.addConstr(BarT == gp.quicksum(gp.quicksum(u*alpha[j,u] for u in range(1, BarW + 1)) for j in J), name='5.1d')
for j in J:
    model.addConstr(gp.quicksum(alpha[j,u] for u in range(1, BarW + 1)) <= 1, name='5.1e'+str(j)+'th')
for i in I:
    model.addConstr(gp.quicksum(y[i,j] for j in J) == 1, name='1.5a'+str(i)+'th')
for i in I:
    for j in J:
        model.addConstr(y[i,j] <= x[j], name='1.5b'+str(i)+','+str(j)+'th')
model.addConstr(gp.quicksum(x[j] for j in J) == BarL, name='1.5c')
model.update()

# Step 2: Epigraphical-type MINLP reformulation

In [44]:
# In this step, we introduce auxiliary nonnegative continuous variables q,r,v (q_{ju}, r_{ju}, v_{ju})

$$\mbox{Add auxiliary nonnegative continuous variables} \ \ q_{ju}, r_{ju}, v_{ju} , j \in J, \ u =1,\ldots,\bar{w} \ \ \mbox{into the model} $$


In [45]:
# At this point, we have polynomial constraints and multilinear objective functions. Those are not readable by Gurobi
# solver. Thus, we do not add them into the model, but reformulate them in the next step.

# Step 3: Quasi-Multilinearization of Polynomial Constraints

In [46]:
# In this step, we do not introduce any auxiliary variables, but start to reformulate the polynomial constraints in Step 2
# Here, we do analytical method or induction method to calculate the coefficients a_{U,j,h},
# for any subset U \subseteq I, |U| \le h, h=1,2,...,BarW + 1 

$$ChoiceOfMethod = 0 \ \ \mbox{do analytical method}, \ = 1 \ \ \mbox{do inductive method}$$

In [47]:
# In this step, we will use a special package in Python, called itertools. This allows us to have any subsets of a set

$$itertools.combinations(U,r) \ \ \mbox{is the list of all subsets of cardinality r of U}$$

In [48]:
# def ChoiceOfMethod(u,h):
#     if h <=4:
#         return 1
#     if h==5:
#         if u>3:
#             return 1
#         else:
#             return 0
#     if h==6:
#         if u>5:
#             return 1
#         else:
#             return 0
#     if h>=7:
#         return 0
# def a(U,j,h):
#     if ChoiceOfMethod(len(U),h)==0:
#         if len(U)> h:
#             return 0
#         else:
#             testt=0
#             L=len(U)
#             for r in range(1,L+1): # r is cardinality of U', r takes any integer value from 1 to |U|
#                 for Uprime in list(itertools.combinations(U,r)): # Take any subset U' |subseteq U, where |U'| = r
#                     QWER = 0
#                     for i in Uprime:
#                         QWER = QWER + lamda[i] * MeanS[i,j]
#                     QWER = QWER ** h # QWER = (\sum_{i \in U'} lamda_{i} E[S_{ij}])^h
#                     testt = testt + ((-1)**(L-r))* QWER 
#             return float(testt)  
#     if ChoiceOfMethod(len(U),h)==1: # Do induction method
#         if len(U)>h:
#             return 0
#         elif h==1:
#             return float(lamda[U] * MeanS[U,j])
#         elif len(U)==h:
#             sum1=0
#             for l in U:
#                 tupleU = tuple(x for x in U if x != l)
#                 sum1 = sum1 + a(tupleU,j,h-1) * lamda[l] * MeanS[l,j]
#             return float(sum1)
#         elif len(U)==1:
#             sum2=0
#             for l in U:
#                 sum2 = sum2 + lamda[l] * MeanS[l,j]
#             return float(sum2 * a(U,j,h-1))
#         else:
#             sum1=0
#             for l in U:
#                 tupleU = tuple(x for x in U if x != l)
#                 sum1 = sum1 + a(tupleU,j,h-1) * lamda[l] * MeanS[l,j]
#             sum2=0
#             for l in U:
#                 sum2 = sum2 + lamda[l] * MeanS[l,j] 
#             return float(sum1 + sum2 * a(U,j,h-1))

In [49]:
def a(U,j,h):
    if len(U)> h:
        return 0
    else:
        testt=0
        L=len(U)
        for r in range(1,L+1): # r is cardinality of U', r takes any integer value from 1 to |U|
            for Uprime in list(itertools.combinations(U,r)): # Take any subset U' |subseteq U, where |U'| = r
                QWER = 0
                for i in Uprime:
                    QWER = QWER + lamda[i] * MeanS[i,j]
                QWER = QWER ** h # QWER = (\sum_{i \in U'} lamda_{i} E[S_{ij}])^h
                testt = testt + ((-1)**(L-r))* QWER 
        return float(testt)  

In [50]:
# def a(U,j,h):
#     if len(U)>h:
#         return 0
#     elif h==1:
#         return float(lamda[U] * MeanS[U,j])
#     elif len(U)==h:
#         sum1=0
#         for l in U:
#             tupleU = tuple(x for x in U if x != l)
#             sum1 = sum1 + a(tupleU,j,h-1) * lamda[l] * MeanS[l,j]
#         return float(sum1)
#     elif len(U)==1:
#         sum2=0
#         for l in U:
#             sum2 = sum2 + lamda[l] * MeanS[l,j]
#         return float(sum2 * a(U,j,h-1))
#     else:
#         sum1=0
#         for l in U:
#             tupleU = tuple(x for x in U if x != l)
#             sum1 = sum1 + a(tupleU,j,h-1) * lamda[l] * MeanS[l,j]
#         sum2=0
#         for l in U:
#             sum2 = sum2 + lamda[l] * MeanS[l,j] 
#         return float(sum1 + sum2 * a(U,j,h-1))

In [51]:
def aConvoluB(U,j,u):
    sum=0
    for k in range(len(U),u+1):
        sum=sum + b[u,k] * a(U,j, k)
    return sum
# abLater=np.zeros(shape=(LengthByUJump0[BarW],n))
# for j in J:
#     q=0
#     for run in ListUJump0:
#         if len(run)>ListuJump0[q]:
#             abLater[q,j]=0
#             q=q+1
#         else:
#             for k in range(len(run),ListuJump0[q]+1):
#                 abLater[q,j] = abLater[q,j] + b[ListuJump0[q],k] * a(run,j, k)
#             q=q+1

In [52]:
# Next, we create spaces to stock values computed, that avoids to re-calculating later. This does not belong to the 
# pseudo-code. The goal is only to reduce the running time.

$$\mbox{This does not belong to the pseudo-code. The goal is just to stock values computed in some spaces}$$

In [53]:
# ListUJump2=[]
# LengthByUJump2=[0]
# ListuJump2=[]
# for u in range(1,BarW+1):
#     for r in range(1,u+3):
#         ListUJump2= ListUJump2 + list(itertools.combinations(I,r))
#         ListuJump2=ListuJump2 + len(list(itertools.combinations(I,r))) * [u]
#     LengthByUJump2.append(int(len(ListUJump2)))
# ListUJump1=[]
# LengthByUJump1=[0]
# ListuJump1=[]
# for u in range(1,BarW+1):
#     for r in range(1,u+2):
#         ListUJump1= ListUJump1 + list(itertools.combinations(I,r))
#         ListuJump1=ListuJump1 + len(list(itertools.combinations(I,r))) * [u]
#     LengthByUJump1.append(int(len(ListUJump1)))
# ListUJump0=[]
# LengthByUJump0=[0]
# ListuJump0=[]
# for u in range(1,BarW+1):
#     for r in range(1,u+1):
#         ListUJump0= ListUJump0 + list(itertools.combinations(I,r))
#         ListuJump0=ListuJump0 + len(list(itertools.combinations(I,r))) * [u]
#     LengthByUJump0.append(int(len(ListUJump0)))
# LengthRJump=[0]
# count=0
# for r in range(1,BarW+3):
#     count=count+len(list(itertools.combinations(I,r)))
#     LengthRJump.append(count)

In [54]:
import time
start_time = time.time()
ListU=[]
for r in range(1,min(4,BarW+1)):
    ListU= ListU + list(itertools.combinations(I,r))
#     Listu=Listu + len(list(itertools.combinations(I,r))) * [r]

In [55]:
# This will be used later. The goal is to reduce the running time.

$$\mbox{Here we stock all the values of } \ \ a_{U,j,u} \ \ \mbox{for} \ \ |U| \le u, \ u=1,\ldots,\bar{w} , \ \ j \in J \ \ \mbox{in aLater}$$

In [56]:
#integratedUju0 = [(U,j,u) for U in ListU for j in J for u in range(1,BarW+1) if len(U)<=u]
integratedUju1 = [(U,j,u) for U in ListU for j in J for u in range(1,BarW+1) if len(U)<=u+1]
#integratedUju2 = [(U,j,u) for U in ListU for j in J for u in range(1,BarW+1) if len(U)<=u+2]
integratedUj = [(U,j) for U in ListU for j in J]

In [57]:
# values_a = {}
# for U in ListU:
#     for j in J:
#         for u in range(1,BarW+1):
#             if len(U) <= u:
#                 value = a(U, j, u)  
#                 values_a[(U, j, u)] = value
# values_aConvoluB = {}
# for U in ListU:
#     for j in J:
#         for u in range(1,BarW+1):
#             if len(U) <= u:
#                 value = aConvoluB(U, j, u)  
#                 values_aConvoluB[(U, j, u)] = value

In [58]:
# aLater=np.zeros(shape=(LengthByUJump0[BarW],n))
# for j in J:
#     q=0
#     for run in ListUJump0:
#         aLater[q,j] = a(run,j,ListuJump0[q]+1)
#         q=q+1

In [59]:
# Next, we calculate the vector b_u = (b_{u,0},...,b_{u,u}). This is a part of pseudo-code

$$\mbox{Compute} \ \ \mathbf{b_u} = (b_{u,0},...,b_{u,u}) \ \ \mbox{see Theorem 5.3}$$

In [60]:
b=np.zeros(shape=(BarW+1, BarW+1))
for u in range(1,BarW+1):
    if u==1:
        b[u,0]=1
        b[u,1]=-1
    elif u==2:
        b[u,0]=4
        b[u,1]=0
        b[u,2]=-1
    else:
        b[u,0]= (u**2) * math.factorial(u-1)
        b[u,1]= (u**2-2*u) * math.factorial(u-1)
        for k in range(2,u):
            term1=(u**2) * math.factorial(u-1) / math.factorial(k)
            term2=(2*u) * math.factorial(u-1) / math.factorial(k-1)
            term3=math.factorial(u-1) / math.factorial(k-2)
            b[u,k]= term1-term2+term3
        b[u,u]=-1

$$\mbox{Here we stock all the values of } \ \ \sum_{k = |U|}^{u} b_{u,k} a_{U,j,k} \ \ \mbox{for} \ \ |U| \le u, \ u=1,\ldots,\bar{w} , \ \ j \in J \ \ \mbox{in abLater}, \ \ \mbox{see 5.75c in the paper}$$

In [61]:
# This will be used later. The goal is to reduce the running time.

In [62]:
# abLater=np.zeros(shape=(LengthByUJump0[BarW],n))
# for j in J:
#     q=0
#     for run in ListUJump0:
#         if len(run)>ListuJump0[q]:
#             abLater[q,j]=0
#             q=q+1
#         else:
#             for k in range(len(run),ListuJump0[q]+1):
#                 abLater[q,j] = abLater[q,j] + b[ListuJump0[q],k] * a(run,j, k)
#             q=q+1

In [63]:
# At this point, step 3 is done, the polynomial constraints at Step 3 can be quasi-multilinearized
# by the coefficients a_{U,j,h} and vectors b_{u}. Quasi-multilinear constraints are still not readable
# by Gurobi solver. Thus, we now moving to Step 4 to reformulate them

# Step 4:  Multilinearization of Quasi-Multilinear Constraints

In [64]:
# In this step, we will multilinearize the constraints in Step 3. To do this,
# we compute the coefficient d^{(1)}, d^{(2)}, d^{(3)}, d^{(4)} (input U,j,u)

$$\mbox{Compute} \ \  d^{(1)}_{U,j,u}, |U| \le u+2  \ \ \mbox{Lemma 5.4}$$

In [65]:
#compute d^{(1)}_{U,j,u}, |U| le u+2 

In [66]:
def d1(U,j,u):
    if len(U)>u+2:
        return 0
    elif len(U)==1:
        sum1=0
        for k in range(1,u+1):
            sum1=sum1+b[u,k]*a(U,j,k)
        return float(lamda[U]**2*(b[u,0] + sum1))
    elif len(U)==2:
        i=U[0]
        t=U[1]
        if u==1:
            sum1=0
            for k in range(1,u+1):
                sum1=sum1+ lamda[i]**2 * b[u,k]*a((t,),j,k)  + lamda[t]**2 * b[u,k]*a((i,),j,k) 
            sum1=sum1 + 2 * b[u,0] * lamda[i] * lamda[t]
            for k in range(1,u+1):
                sum1 = sum1+2 * lamda[i] * lamda[t] * b[u,k] * (a((t,),j,k)+ a((i,),j,k))
            return float(sum1)
        else:
            sum1=0
            for k in range(1,u+1):
                sum1=sum1+ lamda[i]**2 * b[u,k]*a((t,),j,k)  + lamda[t]**2 * b[u,k]*a((i,),j,k)
            sum1=sum1 + 2 * b[u,0] * lamda[i] * lamda[t]
            for k in range(2,u+1):
                sum1=sum1+ (lamda[i]**2+lamda[t]**2)*b[u,k]*a(U,j,k)
            for k in range(1,u+1):
                sum1 = sum1+2 * lamda[i] * lamda[t] * b[u,k] * (a((t,),j,k)+ a((i,),j,k))
            for k in range(2,u+1):
                sum1 = sum1+2 * lamda[i] * lamda[t] * b[u,k] * a(U,j,k)
            return float(sum1)
    else:
        if len(U)==u+2:
            sum1=0
            for (i,t) in list(itertools.combinations(U,2)):
                tupleU = tuple(x for x in U if x != i and x != t)
                sum1 = sum1 + 2 * lamda[i] * lamda[t] * b[u,u] * a(tupleU,j,u)
            return float(sum1)
        elif len(U)==u+1:
            sum1=0
            for (i,t) in list(itertools.combinations(U,2)):
                tupleUi = tuple(x for x in U if x != i)
                tupleUt = tuple(x for x in U if x != t)
                tupleU = tuple(x for x in U if x != i and x != t)
                sum1 = sum1 + 2 * lamda[i] * lamda[t] * b[u,u] * (a(tupleUi,j,u)+a(tupleUt,j,u))
                sum1 = sum1 + 2 * lamda[i] **2 * b[u,u] * a(tupleUi,j,u) + 2 * lamda[t] **2 * b[u,u] * a(tupleUt,j,u)
                sum1 = sum1 + 2 * lamda[i] * lamda[t] * (b[u,u-1] * a(tupleU,j,u-1) + b[u,u] * a(tupleU,j,u))
            return float(sum1)
        else:
            L=len(U)
            sum1=0
            sum2=0
            for k in range(L,u+1):
                sum2=sum2+b[u,k] * a(U,j,k)
            sum3=0
            sum4=0
            sum5=0
            sum6=0
            for i in U:
                tupleUi = tuple(x for x in U if x != i)
                sum5=sum5+lamda[i]**2
                for k in range(L-1,u+1):
                    sum4=sum4+ lamda[i] **2 * b[u,k] * a(tupleUi,j,k)
            for (i,t) in list(itertools.combinations(U,2)):
                tupleUi = tuple(x for x in U if x != i)
                tupleUt = tuple(x for x in U if x != t)
                tupleU = tuple(x for x in U if x != i and x != t)
                sum1=sum1+lamda[i] * lamda[t]
                sum6 = sum6 + lamda[i] * lamda[t] * b[u,L-2] * a(tupleU,j,L-2)
                for k in range(L-1,u+1):
                    sum3=sum3 + lamda[i] * lamda[t] * b[u,k] * (a(tupleUi,j,k)+a(tupleUt,j,k))
                    sum6 = sum6 + lamda[i] * lamda[t] * b[u,k] * a(tupleU,j,k)
            return float(2*sum1*sum2+2*sum3+sum4+sum5 * sum2 + 2 * sum6)                                            

In [67]:
# Next, we stock the values of d^{(1)} in an array, called d11. The advantage is  that will be used several times later, 
# without re-calling the function.

$$\mbox{Here, stock the values of} \ \ d^{(1)}_{U,j,u}, \ \ |U| \le u+2,  \ \ u = 1,\ldots, \bar{w}, \ \ j \in J \ \ \mbox{in d11}$$

In [68]:
# d11=np.zeros(shape=(LengthByUJump2[BarW],n))
# for j in J:
#     q=0
#     for run in ListUJump2:
#         d11[q,j] = d1(run,j,ListuJump2[q])
#         q=q+1

$$\mbox{Compute} \ \  d^{(2)}_{U,j,u}, |U| \le u+1  \ \ \mbox{Lemma 5.5}$$

In [69]:
#compute d^{(2)}_{U,j,u}, |U| le u+1

In [70]:
def d2(U,j,u):
    if len(U)>u+1:
        return 0
    if len(U)==1:
        sum1=0
        for k in range(1,u+1):
            sum1=sum1+b[u,k]*a(U,j,k)
        return float(lamda[U]*(b[u,0] + sum1))
    elif len(U)==u+1:
        L=len(U)
        sum1=0
        for i in U:
            tupleUi = tuple(x for x in U if x != i)
            for k in range(L-1,u+1):
                sum1=sum1+ lamda[i] * b[u,k] * a(tupleUi,j,k)
        return float(sum1)
    else:
        L=len(U)
        sum1=0
        for i in U:
            tupleUi = tuple(x for x in U if x != i)
            sum1=sum1+ lamda[i] * b[u,L-1] * a(tupleUi,j,L-1)
            for k in range(L,u+1):
                sum1=sum1+ lamda[i] * b[u,k] * a(tupleUi,j,k) + lamda[i] * b[u,k] * a(U,j,k)
        return float(sum1)

In [71]:
# Next, we stock the values of d^{(2)} in an array, called d22. The advantage is  that will be used several times later, 
# without re-calling the function.

$$\mbox{Here, stock the values of} \ \ d^{(2)}_{U,j,u}, \ \ |U| \le u+1,  \ \ u = 1,\ldots, \bar{w}, \ \ j \in J \ \ \mbox{in d22}$$

In [72]:
# d22=np.zeros(shape=(LengthByUJump1[BarW],n))
# for j in J:
#     q=0
#     for run in ListUJump1:
#         d22[q,j] = d2(run,j,ListuJump1[q])
#         q=q+1

In [73]:
#compute d^{(3)}_{U,j,u}, |U| le u+2

$$\mbox{Compute} \ \  d^{(3)}_{U,j,u}, |U| \le u+2  \ \ \mbox{Lemma 5.6}$$

In [74]:
def d3(U,j,u):
    if len(U)>u+2:
        return 0
    if len(U)==1:
        return float(lamda[U] * c2T[U,j] * a(U,j,u+1))
    elif len(U)==u+2:
        sum1=0
        for i in U:
            tupleUi = tuple(x for x in U if x != i)
            sum1=sum1+ lamda[i] * c2T[i,j] * a(tupleUi,j,u+1)
        return float(sum1)
    else:
        sum1=0
        for i in U:
            tupleUi = tuple(x for x in U if x != i)
            sum1=sum1+ lamda[i] * c2T[i,j] * a(tupleUi,j,u+1) + lamda[i] * c2T[i,j] * a(U,j,u+1)
        return float(sum1)

In [75]:
# Next, we stock the values of d^{(3)} in an array, called d33. The advantage is  that will be used several times later, 
# without re-calling the function.

$$\mbox{Here, stock the values of} \ \ d^{(3)}_{U,j,u}, \ \ |U| \le u+2,  \ \ u = 1,\ldots, \bar{w}, \ \ j \in J \ \ \mbox{in d33}$$

In [76]:
# d33=np.zeros(shape=(LengthByUJump2[BarW],n))
# for j in J:
#     q=0
#     for run in ListUJump2:
#         d33[q,j] = d3(run,j,ListuJump2[q])
#         q=q+1

In [77]:
#compute d^{(4)}_{U,j,u}, |U| le u

$$\mbox{Compute} \ \  d^{(4)}_{U,j,u}, |U| \le u  \ \ \mbox{Lemma 5.7}$$

In [78]:
def d4(U,j,u):
    if len(U)>u:
        return 0
    if u==1:
        return float(lamda[U]*SecondMS[U,j])
    elif len(U)==1:
        return float(lamda[U] * SecondMS[U,j] * a(U,j,u-1))
    elif len(U)==u:
        sum1=0
        for i in U:
            tupleUi = tuple(x for x in U if x != i)
            sum1=sum1+ lamda[i] * SecondMS[i,j] * a(tupleUi,j,u-1)
        return float(sum1)
    else:
        sum1=0
        for i in U:
            tupleUi = tuple(x for x in U if x != i)
            sum1=sum1+ lamda[i] * SecondMS[i,j] * a(tupleUi,j,u-1) + lamda[i] * SecondMS[i,j] * a(U,j,u-1)
        return float(sum1)

In [79]:
d4((1,2,3,4),4,4)

8.509763901808792e-06

In [80]:
# Next, we stock the values of d^{(4)} in an array, called d44. The advantage is  that will be used several times later, 
# without re-calling the function.

$$\mbox{Here, stock the values of} \ \ d^{(4)}_{U,j,u}, \ \ |U| \le u,  \ \ u = 1,\ldots, \bar{w}, \ \ j \in J \ \ \mbox{in d44}$$

In [81]:
# d44=np.zeros(shape=(LengthByUJump0[BarW],n))
# for j in J:
#     q=0
#     for run in ListUJump0:
#         d44[q,j] = d4(run,j,ListuJump0[q])
#         q=q+1

In [82]:
# We defined all coefficients d^{(1)},  d^{(2)}, d^{(3)}, d^{(4)}. We are now moving to the final step, Linearization

# Step 5:  Linearization and MILP Reformulation

In [83]:
# First, we need to get upper bounds for continuous bounded variables, i.e. q_{ju}, r_{ju}, etc. To get them, we need to
# solve additional optimization problems and/or derive closed-form analytical formulation of these bounds.
# We solve several optimization problems or one optimization problem to get all upper bounds at once
# The bounds we need to get here is Barq_{ju}, Barr_{ju}, Barv_{ju}, for (j,u) \in \mathcal{V}
# In this file.py, we decide to temporarily assign a high deterministic value for this.
# We want the following holds

$$\bar{v}_{ju} \ge (5.3)$$ $$\bar{q}_{ju} \ge \frac{\sum_{l \in I}\lambda_l y_{lj} c^2_{T_{lj}}}{\sum_{l \in I}\lambda_l y_{lj}} \times (5.3)$$  $$\bar{r}_{ju} \ge \frac{\left(\sum_{l \in I} \lambda_l y_{lj}\right)\left(\sum_{l \in I} \lambda_l y_{lj} \mathbb{E}[S_{lj}^2]\right)}{\left(\sum_{l \in I} \lambda_l y_{lj} \mathbb{E}[S_{lj}]\right)^2} \times (5.3)$$

In [84]:
M=500000 # For now, we just take a high value to test the code, M is used as bounds for all q_{ju}, r_{ju}, etc

$$\mbox{Adding decision variables into the model, including} \ \ q, \ r, \ v,  \eta, \ \zeta^{(1)}, \ \zeta^{(2)}, \ \zeta^{(3)}, \omega^{(1)}, \ \omega^{(2)}, \ \omega^{(3)}$$

In [85]:
#qVar=model.addVars(n, BarW + 1, vtype=GRB.CONTINUOUS, lb=0, ub=M, name='qVar') # introduce auxiliary nonnegative vector q in the model
#rVar=model.addVars(n, BarW + 1, vtype=GRB.CONTINUOUS, lb=0, ub=M, name='rVar') # introduce auxiliary nonnegative vector r in the model
vVar=model.addVars(n, BarW + 1, vtype=GRB.CONTINUOUS, lb=0, ub=M, name='vVar') # introduce auxiliary nonnegative vector v in the model
# LEN=LengthByUJump0[BarW]-LengthByUJump0[BarW-1]
eta=model.addVars(integratedUj, vtype=GRB.CONTINUOUS, lb=0,ub=1, name='eta')
#zeta1=model.addVars(integratedUju2, vtype=GRB.CONTINUOUS,lb=0, ub=M, name='zeta1')   
#zeta2=model.addVars(integratedUju0, vtype=GRB.CONTINUOUS,lb=0, ub=M, name='zeta2')
zeta3=model.addVars(integratedUju1, vtype=GRB.CONTINUOUS,lb=0, ub=M, name='zeta3')
#omega1=model.addVars(m, n, BarW + 1, vtype=GRB.CONTINUOUS,lb=0, ub=M, name='omega1')     
#omega2=model.addVars(m, n, BarW + 1, vtype=GRB.CONTINUOUS,lb=0, ub=M, name='omega2')
omega3=model.addVars(m, n, BarW + 1, vtype=GRB.CONTINUOUS,lb=0, ub=M, name='omega3')
model.update()

$$\mbox{Add} \ \ \sum_{U \subseteq I, \ |U| \le u+2} d^{(3)}_{U,j,u} \eta^{}_{U,j} = 2 \sum_{U \subseteq I\ |U| \le u+2} d^{(1)}_{U,j,u} \zeta^{(1)}_{U,j,u}, \quad \quad \quad \forall \ (j,u) \in \mathcal{V} \ \ (5.75b)$$

$$\mbox{Add} \ \ \sum_{U \subseteq I ,\ |U| \le u} d^{(4)}_{U,j,u} \eta^{}_{U,j} = 2\left(b_{u,0} r_{ju}+ \sum_{U \subseteq I, \ |U| \le u} \left(\sum_{k=|U|}^{u}b_{u,k}a^{}_{U,j,k}\right)\zeta^{(2)}_{U,j,u}\right), \quad \quad \quad \forall \ (j,u) \in \mathcal{V} \ \ (5.75c)$$

$$\mbox{Add} \ \ \sum_{U \subseteq I\ |U| \le u+1}a^{}_{U,j,u+1} \eta^{}_{U,j} = 2  \sum_{U \subseteq I\ |U| \le u+1} d^{(2)}_{U,j,u} \zeta^{(3)}_{U,j,u}, \quad \quad \quad \forall \ (j,u) \in \mathcal{V} \ \ (5.75d)$$

In [86]:
# for u in range(1,BarW + 1):
#     for j in J:
#         model.addConstr(gp.quicksum(d33[run,j] * eta[run-LengthByUJump2[u-1],j] for run in range(LengthByUJump2[u-1],LengthByUJump2[u])) == 2 * gp.quicksum(d11[run,j] * zeta1[run,j] for run in range(LengthByUJump2[u-1],LengthByUJump2[u])), name='5.75b'+str(j+1)+str(u))
#         model.addConstr(gp.quicksum(d44[run,j] * eta[run-LengthByUJump0[u-1],j] for run in range(LengthByUJump0[u-1],LengthByUJump0[u])) == 2 * b[u,0] * rVar[j,u] + 2 * gp.quicksum(abLater[run,j] * zeta2[run,j] for run in range(LengthByUJump0[u-1],LengthByUJump0[u])), name='5.75c'+str(j+1)+str(u))             
#         model.addConstr(gp.quicksum(aLater[run,j] * eta[run-LengthByUJump1[u-1],j] for run in range(LengthByUJump1[u-1],LengthByUJump1[u])) == 2 * gp.quicksum(d22[run,j] * zeta3[run,j] for run in range(LengthByUJump1[u-1],LengthByUJump1[u])), name='5.75d'+str(j+1)+str(u))                                                                                                                                                                                                                                                                                                                        

In [87]:

for u in range(1,BarW + 1):
    for j in J:
#        model.addConstr(gp.quicksum(d3(U,j,u) * eta[U,j] for U in ListU if len(U) <= u+2) == 2 * gp.quicksum(d1(U,j,u) * zeta1[U,j,u] for U in ListU if len(U)<=u+2), name='5.75b'+str(j+1)+str(u))
#        model.addConstr(gp.quicksum(d4(U,j,u) * eta[U,j] for U in ListU if len(U) <= u) == 2 * b[u,0] * rVar[j,u] + 2 * gp.quicksum(aConvoluB(U,j,u) * zeta2[U,j,u] for U in ListU if len(U) <=u), name='5.75c'+str(j+1)+str(u))              
        model.addConstr(gp.quicksum(a(U,j,u+1) * eta[U,j] for U in ListU if len(U)<=u+1) == 2 * gp.quicksum(d2(U,j,u) * zeta3[U,j,u] for U in ListU if len(U)<=u+1), name='5.75d'+str(j+1)+str(u))         

$$\mbox{For now, we add the traditional expected-value steady state condition, waiting for the chance constraint section}$$

$$\mbox{Add the expected-value steady state constraint} \ \ \sum_{i \in I} \lambda_i y_{ij} \mathbb{E}[S_{ij}] \le \sum_{u=1}^{\bar{w}}u\alpha_{ju}$$

In [88]:
for j in J:
    model.addConstr(gp.quicksum(lamda[i] * y[i,j] * MeanS[i,j] for i in I) <=gp.quicksum(u * alpha[j,u] for u in range(1,BarW + 1)))

$$\mbox{Add objective function} \ \ \frac{1}{\sum_{l \in I} \lambda_l}\left[ \sum_{i \in I}\sum_{j \in J} t_{ij}\lambda_i y_{ij} + \sum_{i \in I}\sum_{j \in J} \sum_{u=1}^{\bar{w}} \lambda_i\left( \omega^{(1)}_{i,j,u} + \omega^{(2)}_{i,j,u} - \omega^{(3)}_{i,j,u}\right) \right] , \ \ \mbox{which should be a linear function now}$$

In [89]:
model.setObjective(1 / float(sum(lamda)) * (gp.quicksum(t[i,j] * lamda[i] * y[i,j] for i in I for j in J) + gp.quicksum(lamda[i] * (omega3[i,j,u]) for i in I for j in J for u in range(1,BarW + 1))), GRB.MINIMIZE)

$$\mbox{Add optimality-based constraints} \ \ \sum_{u=1}^{\bar{w}}\alpha_{ju} = x_j$$

In [90]:
opti1=model.addConstrs(gp.quicksum(y[i,j] for i in I) >= x[j] for j in J)
for j in J:
    opti1[j].Lazy=3
opti2=model.addConstr(gp.quicksum(alpha[j,u] for j in J for u in range(1,BarW+1)) == BarL)
opti2.Lazy=3
# model.addConstrs(gp.quicksum(alpha[j,u] for u in range(1, BarW + 1)) == x[j] for j in J)
model.update()

$$\mbox{Set} \ \eta^{}_{U,j} = \prod_{k \in U}y_{kj}, \ \ \mbox{which is ensured by} \ \ (\textbf{y}_{U,j}, \eta^{}_{U,j}) \in \mathcal{H}^{\textbf{y}_{U,j}}_{\eta^{}_{U,j}}, \quad \quad \quad \forall \ U \subseteq I, \ |U| \le \bar{w}+2, \ j \in J$$

1) $$\mbox{Add} \ \ (\textbf{y}_{U,j}, \eta^{}_{U,j}) \in \mathcal{H}^{\textbf{y}_{U,j}}_{\eta^{}_{U,j}}, \quad \quad \quad \forall \ U \subseteq I, \ |U| \le \bar{w}+2, \ j \in J \ \ \mbox{into the problem}$$

$$\zeta^{(1)}_{U,j,u} = q_{ju} \prod_{k \in U}y_{kj}, \ \zeta^{(2)}_{U,j,u} = r_{ju} \prod_{k \in U}y_{kj}, \ \zeta^{(3)}_{U,j,u} = v_{ju} \prod_{k \in U}y_{kj}$$

2) $$\mbox{Add} \ \ (\textbf{y}_{U,j}, q_{ju}, \zeta^{(1)}_{U,j,u}) \in \mathcal{Q}^{\textbf{y}_{U,j}}_{q_{ju}, \ \zeta^{(1)}_{U,j,u}} , \quad \quad \quad \forall \ U \subseteq I, \ |U| \le u+2, \ (j,u) \in \mathcal{V} \ \ \mbox{}$$ 

3) $$\mbox{Add} \ \ (\textbf{y}_{U,j}, r_{ju}, \zeta^{(2)}_{U,j,u}) \in \mathcal{Q}^{\textbf{y}_{U,j}}_{r_{ju}, \ \zeta^{(2)}_{U,j,u}} , \quad \quad \quad \forall \ U \subseteq I, \ |U| \le u, \ (j,u) \in \mathcal{V} \ \ \mbox{}$$

4) $$\mbox{Add} \ \ (\textbf{y}_{U,j}, v_{ju}, \zeta^{(3)}_{U,j,u}) \in \mathcal{Q}^{\textbf{y}_{U,j}}_{v_{ju}, \ \zeta^{(3)}_{U,j,u}} , \quad \quad \quad \forall \ U \subseteq I, \ |U| \le u+1, \ (j,u) \in \mathcal{V} \ \ \mbox{}$$ 

$$\omega_{i,j,u}^{(1)} =q_{ju} y_{ij} \alpha_{ju}, \ \omega_{i,j,u}^{(2)} =r_{ju} y_{ij} \alpha_{ju}, \omega_{i,j,u}^{(3)} =v_{ju} y_{ij} \alpha_{ju}$$

5) $$\mbox{Add} \ \ ((y_{ij}, \alpha_{ju}), q_{ju}, \omega^{(1)}_{i,j,u}) \in \mathcal{Q}^{(y_{ij}, \alpha_{ju})}_{q_{ju}, \ \omega^{(1)}_{i,j,u}} , \quad \quad \quad \forall \ i \in I, \ (j,u) \in \mathcal{V} \ \ \mbox{}$$ 

6) $$\mbox{Add} \ \ ((y_{ij}, \alpha_{ju}), r_{ju}, \omega^{(2)}_{i,j,u}) \in \mathcal{Q}^{(y_{ij}, \alpha_{ju})}_{r_{ju}, \ \omega^{(2)}_{i,j,u}} , \quad \quad \quad \forall \ i \in I, \ (j,u) \in \mathcal{V} \ \ \mbox{}$$

7) $$\mbox{Add} \ \ ((y_{ij}, \alpha_{ju}), v_{ju}, \omega^{(3)}_{i,j,u}) \in \mathcal{Q}^{(y_{ij}, \alpha_{ju})}_{v_{ju}, \ \omega^{(3)}_{i,j,u}} , \quad \quad \quad \forall \ i \in I, \ (j,u) \in \mathcal{V} \ \ \mbox{}$$

In [91]:
# 1) 
McCormickEtaA=model.addConstrs(eta[U,j] <= y[i,j] for U in ListU for j in J for i in U)
McCormickEtaB=model.addConstrs(eta[U,j] >= gp.quicksum(y[i,j] for i in U) - len(U) + 1 for U in ListU for j in J)
# for U in ListU:
#     if len(U)<3:
#         for j in J:
#             McCormickEtaB[U,j].Lazy=3
#             for i in U:
#                 McCormickEtaA[U,j,i].Lazy=3
# for (U,j) in integratedUj:
#     McCormickEtaA=model.addConstrs(eta[U,j] <= y[i,j] for i in U)
#     McCormickEtaB=model.addConstr(eta[U,j] >= gp.quicksum(y[i,j] for i in U) - len(U) + 1)
#     for i in U:
#         McCormickEtaA[i].Lazy=2
#     McCormickEtaB.Lazy=2
# for run in range(LengthByUJump0[BarW-1], LengthByUJump0[BarW]):
#     for j in J:
#         McCormickEtaB[run,j].Lazy=3
#         for t in range(BarW+2):
#             if t<len(ListUJump2[run]):
#                 McCormickEtaA[run,j,t].Lazy=3
# model.update()
# 2) 
#McCormickZeta1A=model.addConstrs(zeta1[U,j,u] <= M * y[i,j] for U in ListU for j in J for u in range(1,BarW+1) for i in U)
#McCormickZeta1B=model.addConstrs(zeta1[U,j,u] >= qVar[j,u] - M * (len(U) - gp.quicksum(y[i,j] for i in U)) for U in ListU for j in J for u in range(1,BarW+1) if len(U)<=u+2)
#McCormickZeta1C=model.addConstrs(zeta1[U,j,u] <= qVar[j,u] for U in ListU for j in J for u in range(1,BarW+1))
# for U in ListU:
#     for j in J:
#         for u in range(1,BarW+1):
#             if len(U)<=u+2:
#                 for i in U:
#                     McCormickZeta1A[U,j,u,i].Lazy=0
#                 McCormickZeta1B[U,j,u].Lazy=0
#                 McCormickZeta1C[U,j,u].Lazy=0
# model.update()
# 3) 
#McCormickZeta2A=model.addConstrs(zeta2[U,j,u] <= M * y[i,j] for U in ListU for j in J for u in range(1,BarW+1) for i in U if len(U)<=u)
#McCormickZeta2B=model.addConstrs(zeta2[U,j,u] >= rVar[j,u] - M * (len(U) - gp.quicksum(y[i,j] for i in U)) for U in ListU for j in J for u in range(1,BarW+1) if len(U)<=u)
#McCormickZeta2C=model.addConstrs(zeta2[U,j,u] <= rVar[j,u] for U in ListU for j in J for u in range(1,BarW+1) if len(U)<=u)
# for U in ListU:
#     if len(U)==3:
#         for j in J:
#             for u in range(len(U),BarW+1):
# #                 for i in U:
# #                     McCormickZeta2A[U,j,u,i].Lazy=0
# #                 McCormickZeta2B[U,j,u].Lazy=0
#                 McCormickZeta2C[U,j,u].Lazy=3
# model.update()
# 4) 
#McCormickZeta3A=model.addConstrs(zeta3[U,j,u] <= M * y[i,j] for U in ListU for j in J for u in range(1,BarW+1) for i in U)
McCormickZeta3B=model.addConstrs(zeta3[U,j,u] >= vVar[j,u] - M * (len(U) - gp.quicksum(y[i,j] for i in U)) for U in ListU for j in J for u in range(1,BarW+1) if len(U) <=u+1)
McCormickZeta3C=model.addConstrs(zeta3[U,j,u] <= vVar[j,u] for U in ListU for j in J for u in range(1,BarW+1))
# for U in ListU:
#     for j in J:
#         for u in range(1,BarW+1):
#             if len(U)<=u+1:
#                 for i in U:
#                     McCormickZeta3A[U,j,u,i].Lazy=0
#                 McCormickZeta3B[U,j,u].Lazy=0
#                 McCormickZeta3C[U,j,u].Lazy=0
# model.update()
# 5) 
# McCormickOmega1A=model.addConstrs(omega1[i,j,u] <= M * y[i,j] for i in I for j in J for u in range(1,BarW + 1))
# McCormickOmega1B=model.addConstrs(omega1[i,j,u] <= M * alpha[j,u] for i in I for j in J for u in range(1,BarW + 1))
#McCormickOmega1C=model.addConstrs(omega1[i,j,u] >= qVar[j,u] - M * (2-y[i,j] -  alpha[j,u]) for i in I for j in J for u in range(1,BarW + 1))
# McCormickOmega1D=model.addConstrs(omega1[i,j,u] <= qVar[j,u] for i in I for j in J for u in range(1,BarW + 1))
# for i in I:
#     for j in J:
#         for u in range(1,BarW + 1):
# #           McCormickOmega1A[i,j,u].Lazy=0
#             McCormickOmega1B[i,j,u].Lazy=3
# #           McCormickOmega1C[i,j,u].Lazy=0
#             McCormickOmega1D[i,j,u].Lazy=3
# model.update()
# 6) 
#McCormickOmega2A=model.addConstrs(omega2[i,j,u] <= M * y[i,j] for i in I for j in J for u in range(1,BarW+1))
#McCormickOmega2B=model.addConstrs(omega2[i,j,u] <= M * alpha[j,u] for i in I for j in J for u in range(1,BarW+1))
#McCormickOmega2C=model.addConstrs(omega2[i,j,u] >= rVar[j,u] - M * (2-y[i,j] -  alpha[j,u]) for i in I for j in J for u in range(1,BarW+1))
#McCormickOmega2D=model.addConstrs(omega2[i,j,u] <= rVar[j,u] for i in I for j in J for u in range(1,BarW+1))
#for i in I:
#    for j in J:
#        for u in range(1,BarW+1):
#            McCormickOmega2A[i,j,u].Lazy=3
#            McCormickOmega2B[i,j,u].Lazy=3
#             McCormickOmega2C[i,j,u].Lazy=3
#             McCormickOmega2D[i,j,u].Lazy=0
# model.update()
# 7) 
#McCormickOmega3A=model.addConstrs(omega3[i,j,u] <= M * y[i,j] for i in I for j in J for u in range(1,BarW + 1))
#McCormickOmega3B=model.addConstrs(omega3[i,j,u] <= M * alpha[j,u] for i in I for j in J for u in range(1,BarW + 1))
McCormickOmega3C=model.addConstrs(omega3[i,j,u] >= vVar[j,u] - M * (2-y[i,j] -  alpha[j,u]) for i in I for j in J for u in range(1,BarW + 1))
McCormickOmega3D=model.addConstrs(omega3[i,j,u] <= vVar[j,u] for i in I for j in J for u in range(1,BarW + 1))
# for i in I:
#     for j in J:
#         for u in range(1,BarW + 1):
# #           McCormickOmega3A[i,j,u].Lazy=0
#             McCormickOmega3B[i,j,u].Lazy=3
# #           McCormickOmega3C[i,j,u].Lazy=0
#             McCormickOmega3D[i,j,u].Lazy=3
model.update()

$$\mbox{Define a callback function to add valid inequalities with nested structure}$$

In [92]:
# for j in J:
#     for r in range(1,BarW):
#         for U in list(itertools.combinations(I,r)):
#             setU = set(U)
#             for t in set(I).difference(setU):
#                 Uprime=tuple(sorted(set(tuple(set(U))+(t,))))
#                 cbEtaUp=model.addConstr(eta[U,j] >= eta[Uprime,j])
#                 cbEtaDown=model.addConstr(eta[U,j] - eta[Uprime,j] <= 1 - y[t,j])
#                 cbEtaUp.Lazy=3
#                 cbEtaDown.Lazy=3
# for j in J:
#     for u in range(2,BarW + 1):    
#         for r in range(1,u):
#             for U in list(itertools.combinations(I,r)):
#                 for t in set(I).difference(setU):
#                     Uprime=tuple(sorted(set(tuple(set(U))+(t,))))
#                     cbZeta2Up=model.addConstr(zeta2[U,j,u] >= zeta2[Uprime,j,u])
#                     cbZeta2Down=model.addConstr(zeta2[U,j,u] - zeta2[Uprime,j,u] <= rVar[j,u] - zeta2[(t,),j,u])
#                     cbZeta2Up.Lazy=3
#                     cbZeta2Down.Lazy=3        

In [93]:
# for j in J:
#     for k in range(1,BarW):
#         for runL in range(LengthByUJump0[BarW-1]+LengthRJump[k-1], LengthByUJump0[BarW-1]+LengthRJump[k]):
#             countnested=0
#             S_k=set(ListUJump0[runL])
#             for runU in range(LengthByUJump0[BarW-1]+LengthRJump[k], LengthByUJump0[BarW-1]+LengthRJump[k+1]):
#                 S_kPlus1=set(ListUJump0[runU])
#                 if S_k.issubset(S_kPlus1)==True:
#                     countnested=countnested+1
#                     i=list(S_kPlus1.difference(S_k))[0]
#                     a=model.addConstr(eta[runL-LengthByUJump0[BarW-1],j] >= eta[runU-LengthByUJump0[BarW-1],j])
#                     b=model.addConstr(eta[runL-LengthByUJump0[BarW-1],j] - eta[runU-LengthByUJump0[BarW-1],j] <= 1 - y[i,j])
#                     a.Lazy=3
#                     b.Lazy=3
#                     if countnested==m-k:
#                         break
# for j in J:
#     for u in range(2,BarW + 1):    
#         for k in range(1,u):
#             for runL in range(LengthByUJump0[u-1]+LengthRJump[k-1], LengthByUJump0[u-1]+LengthRJump[k]):
#                 countnested=0
#                 S_k=set(ListUJump0[runL])
#                 for runU in range(LengthByUJump0[u-1]+LengthRJump[k], LengthByUJump0[u-1]+LengthRJump[k+1]):
#                     S_kPlus1=set(ListUJump0[runU])
#                     if S_k.issubset(S_kPlus1)==True:
#                         countnested=countnested+1
#                         i=list(set(S_kPlus1).difference(set(S_k)))[0]
#                         a=model.addConstr(zeta2[runL,j] >= zeta2[runU,j])
#                         b=model.addConstr(zeta2[runL,j] - zeta2[runU,j] <= rVar[j,u] - zeta2[LengthByUJump0[u-1]+i,j])
#                         a.Lazy=3
#                         b.Lazy=3
#                         if countnested==m-k:
#                             break

In [94]:
def mycallback(model, where):
#     if where == GRB.Callback.MIPNODE:
    if where == GRB.Callback.MIPSOL:
#         status = model.cbGet(GRB.Callback.MIPNODE_STATUS)
#         if status == GRB.OPTIMAL:
        if model.cbGet(GRB.Callback.MIPSOL_NODCNT) == 0:
            rel_y = model.cbGetNodeRel(y)
            rel_eta = model.cbGetNodeRel(eta)
#            rel_q = model.cbGetNodeRel(qVar)
#            rel_r = model.cbGetNodeRel(rVar)
            rel_v = model.cbGetNodeRel(vVar)
#            rel_zeta1 = model.cbGetNodeRel(zeta1)
#            rel_zeta2 = model.cbGetNodeRel(zeta2)
            rel_zeta3 = model.cbGetNodeRel(zeta3)
#            rel_omega1 = model.cbGetNodeRel(omega1)
#            rel_omega2 = model.cbGetNodeRel(omega2)
            rel_omega3 = model.cbGetNodeRel(omega3)
            rel_alpha = model.cbGetNodeRel(alpha)
            for j in J:
                for r in range(1,2):
                    for U in list(itertools.combinations(I,r)):
                        setU = set(U)
                        for t in set(I).difference(setU):
                            Uprime=tuple(sorted(set(tuple(set(U))+(t,))))
                            if rel_eta[U,j] < rel_eta[Uprime,j]:
                                model.cbCut(eta[U,j] >= eta[Uprime,j])
                            if rel_eta[U,j] - rel_eta[Uprime,j] > 1 - rel_y[t,j]:
                                model.cbCut(eta[U,j] - eta[Uprime,j] <= 1 - y[t,j])
#             for j in J:
#                 for u in range(1,BarW + 1):    
#                     for r in range(1,u+2):
#                         for U in list(itertools.combinations(I,r)):
#                             for t in set(I).difference(setU):
#                                 Uprime=tuple(sorted(set(tuple(set(U))+(t,))))
#                                 if rel_zeta1[U,j,u] < rel_zeta1[Uprime,j,u]:
#                                     model.cbCut(zeta1[U,j,u] >= zeta1[Uprime,j,u])
#                                 if rel_zeta1[U,j,u] - rel_zeta1[Uprime,j,u] > rel_q[j,u] - rel_zeta1[(t,),j,u]:
#                                     model.cbCut(zeta1[U,j,u] - zeta1[Uprime,j,u] <= qVar[j,u] - zeta1[(t,),j,u])
#            for j in J:
#                for u in range(2,3):    
#                    for r in range(1,u):
#                        for U in list(itertools.combinations(I,r)):
#                            for t in set(I).difference(setU):
#                                Uprime=tuple(sorted(set(tuple(set(U))+(t,))))
#                                if rel_zeta2[U,j,u] < rel_zeta2[Uprime,j,u]:
#                                    model.cbCut(zeta2[U,j,u] >= zeta2[Uprime,j,u])
#                                if rel_zeta2[U,j,u] - rel_zeta2[Uprime,j,u] > rel_r[j,u] - rel_zeta2[(t,),j,u]:
#                                    model.cbCut(zeta2[U,j,u] - zeta2[Uprime,j,u] <= rVar[j,u] - zeta2[(t,),j,u])
            for j in J:
                for u in range(1,2):    
                    for r in range(1,u+1):
                        for U in list(itertools.combinations(I,r)):
                            for t in set(I).difference(setU):
                                Uprime=tuple(sorted(set(tuple(set(U))+(t,))))
                                if rel_zeta3[U,j,u] < rel_zeta3[Uprime,j,u]:
                                     model.cbCut(zeta3[U,j,u] >= zeta3[Uprime,j,u])
                                if rel_zeta3[U,j,u] - rel_zeta3[Uprime,j,u] > rel_v[j,u] - rel_zeta3[(t,),j,u]:
                                     model.cbCut(zeta3[U,j,u] - zeta3[Uprime,j,u] <= vVar[j,u] - zeta3[(t,),j,u])


In [95]:
# LazyEta1 = model.addConstrs(eta[U,j] >= eta[tuple(sorted(set(tuple(set(U))+(t,)))),j] for j in J for r in range(1,2) for U in list(itertools.combinations(I,r)) for t in set(I).difference(set(U)))
# LazyEta2 = model.addConstrs(eta[U,j] - eta[tuple(sorted(set(tuple(set(U))+(t,)))),j] <= 1 - y[t,j] for j in J for r in range(1,2) for U in list(itertools.combinations(I,r)) for t in set(I).difference(set(U)))
# for j in J:
#     for r in range(1,2):
#         for U in list(itertools.combinations(I,r)):
#             for t in set(I).difference(set(U)):
#                 LazyEta1[j,r,U,t].Lazy=3
#                 LazyEta2[j,r,U,t].Lazy=3
# LazyZeta21=model.addConstrs(zeta2[U,j,u] >= zeta2[tuple(sorted(set(tuple(set(U))+(t,)))),j,u] for j in J for u in range(2,3) for r in range(1,u) for U in list(itertools.combinations(I,r)) for t in set(I).difference(set(U)))
# LazyZeta22=model.addConstrs(zeta2[U,j,u] - zeta2[tuple(sorted(set(tuple(set(U))+(t,)))),j,u] <= rVar[j,u] - zeta2[(t,),j,u] for j in J for u in range(2,3) for r in range(1,u) for U in list(itertools.combinations(I,r)) for t in set(I).difference(set(U)))
# for j in J:
#     for u in range(2,3):    
#         for r in range(1,u):
#             for U in list(itertools.combinations(I,r)):
#                 for t in set(I).difference(set(U)):
#                     LazyZeta21[j,u,r,U,t].Lazy=3
#                     LazyZeta22[j,u,r,U,t].Lazy=3

In [96]:
model.setParam('NumericFocus', 3)
model.setParam(GRB.Param.TimeLimit, 3600 * 1)
#model.setParam(GRB.Param.Threads, 1)
model.setParam(GRB.Param.PreCrush, 1)
# model.setParam('Method', 4)
# model.params.BarHomogeneous = 1
# model.Params.Presolve = 2
# model.Params.PresolveTol = 1e-6
# model.Params.PresolveLinear = 1  
# model.Params.PresolveQuadratic = 0
# model.Params.Threads = 8 

Set parameter NumericFocus to value 3
Set parameter TimeLimit to value 3600
Set parameter PreCrush to value 1


In [97]:
# model.Params.LazyConstraints = 1

In [98]:
model.optimize(mycallback)

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  3600
NumericFocus  3
PreCrush  1

Academic license 2471257 - for non-commercial use only - registered to ho___@gwu.edu
Optimize a model with 178888 rows, 77785 columns and 586830 nonzeros
Model fingerprint: 0xe748e17a
Variable types: 77265 continuous, 520 integer (520 binary)
Coefficient statistics:
  Matrix range     [3e-06, 5e+05]
  Objective range  [1e-05, 1e-01]
  Bounds range     [1e+00, 5e+05]
  RHS range        [1e+00, 1e+06]
Presolve removed 1010 rows and 1010 columns
Presolve time: 3.63s
Presolved: 177878 rows, 76775 columns, 660045 nonzeros
Extracted 6 lazy constraints
Variable types: 76260 continuous, 515 integer (515 binary)

Deterministic concurrent LP optimizer: primal simplex, dual s

In [99]:
# Val_X=[]
# for var in x:
#     Val_X.append(int(x[var].X))
# Val_Y=np.zeros(shape=(m,n))
# for var in y:
#     i=var[0]
#     j=var[1]
#     Val_Y[i,j]=int(round(y[var].X,1))
# Val_alpha=np.zeros(shape=(n, BarW + 1))
# for var in alpha:
#     j=var[0]
#     u=var[1]
#     if u > 0:
#         Val_alpha[j,u]=int(round(alpha[var].X,1))
# Val_omega2=np.zeros(shape=(m,n,BarW+1))
# for var in omega2:
#     i=var[0]
#     j=var[1]
#     u=var[2]
#     if u > 0:
#         Val_omega2[j,u]=omega2[var].X

In [100]:
facility

,id_bases,type,street,latitude,longitude,x_bases,y_bases,z_bases
0,1,Fire Station,1468 NIMMO PKWY,36.764154,-76.023000,1234.10032,-4958.35519,3817.28814
1,17,Fire Station,595 PRINCESS ANNE RD,36.603012,-76.026976,1236.38530,-4968.77418,3802.97488
2,7,Fire Station,6009 BLACKWATER RD,36.584900,-76.082865,1231.78363,-4971.19052,3801.31002
3,4,Fire Station,5656 PROVIDENCE RD,36.813926,-76.192077,1218.70816,-4958.71642,3821.76123
4,6,Fire Station,949 S BIRDNECK RD,36.810987,-75.992934,1235.98553,-4954.66183,3821.47132


In [101]:
customer_loc

,id_ohca,receivedtime,minimumresponsetime,latitude,longitude,incident_location,x_ohca,y_ohca,z_ohca,demand_year,demand_month,mon_yr,qrt_yr,occurence
0,393,2019-04-25 22:56:00,8.933333,36.801767,-76.077191,3200 SCARBOROUGH WAY,1228.841526,-4957.055033,3820.671575,2019,4,2019-04,2019-2,1
1,450,2018-07-05 18:48:00,12.066667,36.833051,-75.970992,400 ATLANTIC AV,1237.521526,-4952.744165,3823.459445,2018,7,2018-07,2018-3,1
2,671,2019-05-18 11:58:00,0.000000,36.809603,-76.199786,5900 INDIAN RIVER RD,1218.107522,-4959.165518,3821.369986,2019,5,2019-05,2019-2,1
3,274,2018-10-20 08:43:00,0.000000,36.772649,-76.036235,2300 WALLINGTON WAY,1232.852992,-4958.059065,3818.075704,2018,10,2018-10,2018-4,1
4,38,2019-01-01 18:53:00,0.000000,36.808763,-76.203283,1000 BRYCE LA,1217.818201,-4959.294263,3821.295121,2019,1,2019-01,2019-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,390,2019-04-02 00:12:00,0.000000,36.792380,-76.080698,3200 HOLLAND RD,1228.688672,-4957.737758,3819.834831,2019,4,2019-04,2019-2,1
96,634,2018-11-25 01:49:00,0.000000,36.771198,-76.185797,5600 FREEWILL LA,1219.929628,-4961.354241,3817.946321,2018,11,2018-11,2018-4,1
97,761,2018-09-09 02:39:00,4.966667,36.806447,-76.141340,900 DUNHILL DR,1223.216026,-4958.124736,3821.088705,2018,9,2018-09,2018-3,2
98,137,2019-01-12 13:44:00,10.366667,36.792097,-76.004683,1400 DOVE DR,1235.269642,-4956.121587,3819.809603,2019,1,2019-01,2019-1,1


# The Queuing Network Design Problem

In [102]:
# print("There are ", n , " candidates facility locations, ", m , " customer locations. The total number of drones is ", BarT , " and the maximum capacity at each facility location is ", BarW , "")  

# The optimal location-allocation strategy is solved and given as follows :

# Optimal location

In [103]:
# print("Set up drone bases at the following locations ", end = '')
# for j in J:
#     if Val_X[j] == 1:
#         print(j+1 , "," , end = '')

# Optimal allocation

In [104]:
# for j in J:
#     print("Facility location ", j+1 , "support the following customers locations :" , end='')
#     for i in I:
#         if Val_Y[i,j]==1:
#             print(i+1 , "," , end='')
#     print()
#     print()

# Optimal Capacity

In [105]:
# Val_W=np.zeros(n)
# for j in J:
#     for u in range(BarW+1):
#         if Val_alpha[j,u]==1:
#             Val_W[j]=u
#             print("Capacity (number of drones) at Facility Location " +str(j+1)+ " is" ,u)
# print("Total number of drones is", BarT)

# Optimal Average Response Time

In [106]:
# optimal_value = model.objVal
# print("Optimal Average Response Time of the Network is ", optimal_value * 60, "mins")
end_time = time.time()
# Calculate the loading constraints time
loading_time = end_time - start_time
print("The loading constraints time using both analytical - inductive method is :", loading_time, " seconds")

The loading constraints time using both analytical - inductive method is : 89.90616178512573  seconds
